In [ ]:
import numpy as np 
from matplotlib import pyplot as plt
import pandas 
import math

if not 'CSV_INPUT' in locals():
    raise Exception("Set `CSV_INPUT` variable")

In [ ]:
data = pandas.read_csv(CSV_INPUT)

data[["rx-hotpink", "ry-hotpink", "rx-lightorange", "ry-lightorange", "rx-green", "ry-green"]] /= 100

plt.figure(figsize=(8, 8), dpi=80)
plt.plot(data["rx-hotpink"], -1 * data["ry-hotpink"])
plt.plot(data["rx-lightorange"], -1 * data["ry-lightorange"])
plt.plot(data["rx-green"], -1 * data["ry-green"])
plt.title("Raw position path")

Raw positions plotted into one graph. Positions of light orange marker(the weight) is not consistent as tracking program was capturing incorrect possition when it was being dropped. 

In [ ]:
DROP_FRAME_AMOUNT=50
SAMPLE_FRAME_AMOUNT=60

weightDroppedFrame = data["rx-lightorange"].first_valid_index()

before = data[:weightDroppedFrame][-SAMPLE_FRAME_AMOUNT:].loc[:]
after = data[weightDroppedFrame + DROP_FRAME_AMOUNT:-DROP_FRAME_AMOUNT].loc[:]

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
axs[0].plot(before["rx-hotpink"], -1 * before["ry-hotpink"])
axs[0].plot(before["rx-lightorange"], -1 * before["ry-lightorange"])
axs[0].plot(before["rx-green"], -1 * before["ry-green"])
axs[0].set_title("Truncated position path before adding weight")
axs[1].plot(after["rx-hotpink"], -1 * after["ry-hotpink"])
axs[1].plot(after["rx-lightorange"], -1 * after["ry-lightorange"])
axs[1].plot(after["rx-green"], -1 * after["ry-green"])
axs[1].set_title("Truncated position path after adding weight")

Nice and clean now!

Now find center, interia of mass, and more

In [ ]:
def distance(a, b):
    return math.sqrt(np.sum(np.square(np.array(a) - np.array(b))))

def uncertainty(a):
    return np.std(a) / math.sqrt(len(a))

I_l = 9.7 * 10 ** -4
m_weight = 20 / 1000

center = np.average(data["rx-green"].dropna()), np.average(data["ry-green"].dropna())
center_x, center_y = np.average(data["rx-green"].dropna()), np.average(data["ry-green"].dropna())

before["r_l"] = np.sqrt((before["rx-hotpink"] - center_x) ** 2 + (before["ry-hotpink"] - center_y) ** 2)
after["r_l"] = np.sqrt((after["rx-hotpink"] - center_x) ** 2 + (after["ry-hotpink"] - center_y) ** 2)
after["r_weight"] = np.sqrt((after["rx-lightorange"] - center_x) ** 2 + (after["ry-lightorange"] - center_y) ** 2)
after["I_weight"] = m_weight * after["r_weight"] ** 2 + 1.32 * 10 ** -6

diff_after = after.diff().fillna(1)
diff_before = before.diff().fillna(1)

diff_before = diff_before.drop(diff_before[diff_before["timestamp"] == 0].index)

before["v_tan"] = np.sqrt(diff_before["rx-hotpink"] ** 2 + diff_before["ry-hotpink"] ** 2) / diff_before["timestamp"] * 1000
before["w"] = before["v_tan"] / before["r_l"]
after["v_tan"] = np.sqrt(diff_after["rx-hotpink"] ** 2 + diff_after["ry-hotpink"] ** 2) / diff_after["timestamp"] * 1000
after["w"] = after["v_tan"] / after["r_l"]

before, after = before.iloc[1:-1] , after.iloc[1:-1]

w_before, w_before_uncert = np.average(before["w"].dropna()), uncertainty(before["w"].dropna())
w_after, w_after_uncert = np.average(after["w"].dropna()), uncertainty(after["w"].dropna())
I_weight = np.average(after["I_weight"])

print("Avg angular velocity before: ", w_before, ". uncertainty: ", uncertainty(before["w"].dropna()))
print("Avg angular velocity after: ", w_after, ". uncertainty: ", uncertainty(after["w"].dropna()))
print("Avg intertia of the weight: ", I_weight)

I_o = (w_after * I_weight) / (w_before - w_after)
I_o_uncert = (w_after_uncert * I_weight) / (w_before - w_after)

print("Average intertial of the object: ", I_o, ". uncertainty: ", I_o_uncert)